In [12]:
import numpy as np
import pandas as pd
from sklearn import svm

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score

In [2]:
import sys
sys.path.insert(0, '../libs/')
%matplotlib inline

import FeatureExtraction
from lxml import etree

training_dataset = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml'
test_dataset = '../../dataset/test/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml'

sexual_predator_ids_file = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt'

chat_based_features_csv_train='../../csv/chat_based_features_training.csv'
chat_based_features_csv_test='../../csv/chat_based_features_test.csv'

In [3]:
training_tree = etree.parse(training_dataset)

In [4]:
autor_conversation_dictionay = FeatureExtraction.extract_author_conversation_node_dictionary_from_XML(training_tree)
sexual_predator_ids = FeatureExtraction.sexual_predator_ids(sexual_predator_ids_file)

In [5]:
train_frame=pd.DataFrame(pd.read_csv(chat_based_features_csv_train))
test_frame=pd.DataFrame(pd.read_csv(chat_based_features_csv_test))

In [6]:

to_excl=['autor','mean time of messages sent','is sexual predator']
features=['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']
X_train=train_frame.ix[:,features]
y_train=np.ravel(train_frame[[-1]])
X_test=test_frame.ix[:,features]
y_test=np.ravel(test_frame[[-1]])
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
X_train.ix[0,features]

((97689, 5), (97689L,), (218702, 5), (218702L,))


number of conversation                            1.000000
percent of conversations started by the author    0.000000
number of messages sent                           2.000000
average percent of lines in conversation          0.666667
number of characters sent by the author           5.000000
Name: 0, dtype: float64

In [7]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [8]:

svmc=svm.SVC()
linearsvm=svm.LinearSVC(max_iter=100000)
linearsvm

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=100000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [10]:
classifier=svmc
classifier.fit(X_train,y_train)
predicted_train=classifier.predict(X_train)
predicted_test=classifier.predict(X_test)


In [ ]:
print(predicted_test[predicted_test==1])

In [15]:
print("train statistics")
print('acc',accuracy_score(y_train,predicted_train))
print('rec',recall_score(y_train,predicted_train))
print('prec',precision_score(y_train,predicted_train))
print('f1',f1_score(y_train,predicted_train))

print("\ntest statistics")
print('acc',accuracy_score(y_test,predicted_test))
print('rec',recall_score(y_test,predicted_test))
print('prec',precision_score(y_test,predicted_test))
print('f1',f1_score(y_test,predicted_test))
print('f0.5',fbeta_score(y_test,predicted_test,0.5))

train statistics
('acc', 0.99890468732405902)
('rec', 0.352112676056338)
('prec', 0.76923076923076927)
('f1', 0.48309178743961356)

test statistics
('acc', 0.99905350659801917)
('rec', 0.23228346456692914)
('prec', 0.83098591549295775)
('f1', 0.36307692307692307)
('f0.5', 0.54832713754646845)


In [ ]:
from sklearn import linear_model
lrc=linear_model.LogisticRegression(n_jobs=8,max_iter=1000000,penalty='l1')
lrc.fit(X,y)